In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow import keras
from typing import Tuple, Any, Dict, List
import json
import pandas as pd

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Mounted at /content/drive
Num GPUs Available:  0


In [ ]:
SEED: int = 123
BATCH_SIZE: int = 32
VAL_SPLIT: float = .2
INPUT_SHAPE: Tuple[int] = (128, 128, 3)
IMAGES_PATH: str = "/content/drive/MyDrive/STUDIA/WDSN/Project/images/processed"
LABELS_PATH: str = "/content/drive/MyDrive/STUDIA/WDSN/Project/images/labels/labels.json"
IMAGE_COUNT: int = len(os.listdir(IMAGES_PATH))

In [ ]:
dataset: pd.DataFrame = pd.read_json(LABELS_PATH).T
dataset["filename"] = dataset.index

In [ ]:
def compare_history(
    histories: List[Dict[str, Any]], 
    labels: List[str], 
    epochs: List[int], 
    metric: str = 'accuracy'
    ) -> None:
    plt.figure()
    for history, label, epoch in zip(histories, labels, epochs):
        plt.plot(range(epoch), history.history['acc'], label=label+" train")
        plt.plot(range(epoch), history.history['val_acc'], label=label+" validation")
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.grid()
    plt.legend()
    plt.show()


def decode_img(path: str, filename: str) -> tf.Tensor:
    full_path: str = os.path.join(IMAGES_PATH, filename)
    img = tf.io.decode_jpeg(tf.io.read_file(full_path), channels=3)
    return tf.image.resize(img, INPUT_SHAPE[:2])


In [ ]:
images = tf.keras.utils.image_dataset_from_directory(
    IMAGES_PATH,
    labels=None,
    label_mode=None,
    image_size=INPUT_SHAPE[:2]
).cache().prefetch(buffer_size=tf.data.AUTOTUNE)#from_tensor_slices([decode_img(IMAGES_PATH, filename) for filename in os.listdir(IMAGES_PATH)])
means: List[float] = tf.data.Dataset.from_tensor_slices([dataset[dataset.index == filename]["mean"] for filename in os.listdir(IMAGES_PATH)]).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
weights: List[float] = tf.data.Dataset.from_tensor_slices([dataset[dataset.index == filename]["weight"] for filename in os.listdir(IMAGES_PATH)]).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 2025 files belonging to 1 classes.


In [ ]:
ds: tf.data.Dataset = tf.data.Dataset.zip((images, means, weights)).shuffle(SEED)#.batch(BATCH_SIZE)
train_dataset = ds.skip(int(IMAGE_COUNT*VAL_SPLIT))#.cache()#.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = ds.take(int(IMAGE_COUNT*VAL_SPLIT))#.cache()#.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
model: keras.models = keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=INPUT_SHAPE, batch_size=BATCH_SIZE),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(
    optimizer='adam',
    loss = tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.losses.MeanSquaredError()],
)

In [ ]:
history = model.fit(
    images, 
    means,
    epochs=10, 
    sample_weight = weights
    )

ValueError: ignored